In [1]:
import osmnx as ox
import pandas as pd
import folium

In [2]:
# Define the place to search (Lisbon, Portugal)
place_name = "Lisbon, Portugal"

In [ ]:
# get all building footprints in some neighborhood
tags = {"building": True}
gdf = ox.features.features_from_place(place_name, tags)
gdf.shape

In [ ]:
fig, ax = ox.plot.plot_footprints(gdf, figsize=(3, 3))


In [ ]:
# get all the parks in some neighborhood
# constrain acceptable `leisure` tag values to `park`
tags = {"leisure": "park"}
gdf = ox.features.features_from_place(place_name, tags)
gdf.shape

In [ ]:
fig, ax = ox.plot.plot_footprints(gdf, figsize=(3, 3))

In [ ]:
tags = {"public_transport": "stop_position"}
gdf = ox.features.features_from_place(place_name, tags)
gdf.shape

In [ ]:
gdf.columns

In [10]:
feature_points = gdf.representative_point()

In [ ]:
m = folium.Map(location=[38.740, -9.150], zoom_start=12.2)  # Centered on Lisbon

# Add clusters to the map
for _, row in gdf[~gdf.subway.isna()].iterrows():

    folium.CircleMarker(
        location=(row['geometry'].y, row['geometry'].x),
        radius=5,
        fill=True,
        fill_opacity=0.6,
    ).add_to(m)

m

In [ ]:
m = folium.Map(location=[38.740, -9.150], zoom_start=12.2)  # Centered on Lisbon

# Add clusters to the map
for _, row in gdf[~gdf.train.isna()].iterrows():

    folium.CircleMarker(
        location=(row['geometry'].y, row['geometry'].x),
        radius=5,
        fill=True,
        fill_opacity=0.6,
    ).add_to(m)

m

In [ ]:
m = folium.Map(location=[38.740, -9.150], zoom_start=12.2)  # Centered on Lisbon

# Add clusters to the map
for _, row in gdf[~gdf.tram.isna()].iterrows():

    folium.CircleMarker(
        location=(row['geometry'].y, row['geometry'].x),
        radius=5,
        fill=True,
        fill_opacity=0.6,
    ).add_to(m)

m

In [51]:
metro = gdf[~gdf.subway.isna()]

In [52]:
metro = metro.drop_duplicates(subset='name', keep='first')

In [ ]:
len(metro.name.unique())

In [ ]:
m = folium.Map(location=[38.740, -9.150], zoom_start=12.2)  # Centered on Lisbon

# Add clusters to the map
for _, row in metro.iterrows():

    folium.CircleMarker(
        location=(row['geometry'].y, row['geometry'].x),
        radius=5,
        color='red',
        fill=True,
        fill_opacity=0.6,
    ).add_to(m)

m

In [28]:
train = gdf[~gdf.train.isna()]

In [ ]:
train.info()

In [30]:
train = train.drop_duplicates(subset='name', keep='first')

In [ ]:
train.info()

In [31]:
trams = gdf[~gdf.tram.isna()]

In [ ]:
trams.info()

In [33]:
trams = trams.drop_duplicates(subset='name', keep='first')

In [ ]:
trams.info()